In [26]:
import Orange
import numpy as np
import pandas as pd

In [27]:
from Orange.data import Table
def sample_data(data, samples_per_class = 20):
    for i, value in enumerate(np.unique(data.Y)):
        indices, = np.where(data.Y == value)
        data_per_class = data[indices]
        sampled_data_class = data_per_class[np.random.choice(len(data_per_class), size=samples_per_class, replace=False)]
        if i == 0:
            sampled_data = sampled_data_class
        else:
            sampled_data = Table.concatenate((sampled_data, sampled_data_class))
    return sampled_data

In [28]:
from Orange.data import Table, Domain, ContinuousVariable, DiscreteVariable
def load_data(number_features = 1, error = 1):
    world = pd.read_csv(f'../data/{number_features}_feature/{number_features}feat_0seed_{error}err_obs-unc-tru.csv')
    X = np.column_stack([world[f"Observed Value {i+1}"] for i in range(number_features)])
    Y = np.array(world["Class"])
    M = np.column_stack([world[f"Uncertainty {i+1}"] for i in range(number_features)])
    Xtv = np.column_stack([world[f"True Value {i+1}"] for i in range(number_features)])
    
    domain = Domain(
        attributes = [ContinuousVariable(f"Observed Value {i+1}") for i in range(number_features)],
        class_vars = DiscreteVariable("Class", values=[str(i) for i in range(max(Y+1))]),
        metas = [ContinuousVariable(f"Uncertainty {i+1}") for i in range(number_features)]
    )
    data = Table.from_numpy(domain, X=X, Y=Y, metas=M)
    
    domain = Domain(
        attributes = [ContinuousVariable(f"Observed Value {i+1}") for i in range(number_features)],
        class_vars = DiscreteVariable("Class", values=[str(i) for i in range(max(Y+1))])
    )
    # X = tv1.reshape(-1, 1)
    test_data = Table.from_numpy(domain, X=Xtv, Y=Y)
    return data, test_data
    

In [29]:
data, test_data = load_data(3)
data

[[-0.420057, 0.58984, -1.39719 | 0] {0.0311298, 0.87465, 0.999956},
 [-0.81707, 0.163862, -0.272897 | 0] {0.361953, 0.740208, 0.0812085},
 [-1.46467, 0.821579, -0.838727 | 0] {0.902254, 0.929806, 0.80239},
 [-0.965303, -0.304932, -0.385335 | 0] {0.348744, 0.0966572, 0.0857702},
 [-0.440521, 0.485449, -1.19095 | 0] {0.00856598, 0.559853, 0.261984},
 ...
]

In [30]:
test_data

[[-0.451186, -0.284811, -0.397237 | 0],
 [-0.455117, -0.576345, -0.354106 | 0],
 [-0.562413, -0.108227, -0.0363372 | 0],
 [-0.616558, -0.208275, -0.471105 | 0],
 [-0.431955, -0.0744034, -0.928964 | 0],
 ...
]

In [31]:
from trees.util.uncertaintreelearner import UncertainTreeLearner
from trees.util.uncertainforestutil import UncertainForestLearner
from trees.owtree import ClassificationTreeLearner

In [32]:
post_hoc = UncertainTreeLearner(max_depth=None, min_samples_leaf=2, min_samples_split=4, sufficient_majority=0.90, post_hoc=True)
forward = UncertainTreeLearner(max_depth=None, min_samples_leaf=2, min_samples_split=4, sufficient_majority=0.90, post_hoc=False)
skl_tree = ClassificationTreeLearner(max_depth=None, min_samples_leaf=2, min_samples_split=4, sufficient_majority=0.90)
uncertain_forest10 = UncertainForestLearner(n_trees=10)
uncertain_forest100 = UncertainForestLearner(n_trees=100)

In [17]:
from Orange.evaluation import TestOnTestData
from Orange.evaluation.scoring import CA
rep = 33
for i in range(3):
    i += 1
    for err in [0.1, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6]:
        data, test_data = load_data(i, error=err)
        average_CA = np.array([0, 0, 0, 0, 0], dtype="float64")
        for _ in range(rep):
            res = TestOnTestData(data=sample_data(data, 20), test_data=test_data, learners=[skl_tree, forward, post_hoc, uncertain_forest10, uncertain_forest100])
            average_CA += CA(res)
        print(f"""{i} Feature, Error {err}
skl_tree  {round(average_CA[0]/rep, 3)}
forward   {round(average_CA[1]/rep, 3)} *min number of instances in leaf no work
post_hoc  {round(average_CA[2]/rep, 3)}
forest10  {round(average_CA[3]/rep, 3)}
forest100 {round(average_CA[4]/rep, 3)}""")    
    print("-----------")

1 Feature, Error 0.1
skl_tree  0.976
forward   0.976 *min number of instances in leaf no work
post_hoc  0.976
forest10  0.975
forest100 0.977
1 Feature, Error 0.2
skl_tree  0.96
forward   0.961 *min number of instances in leaf no work
post_hoc  0.962
forest10  0.958
forest100 0.96
1 Feature, Error 0.4
skl_tree  0.905
forward   0.938 *min number of instances in leaf no work
post_hoc  0.94
forest10  0.946
forest100 0.952
1 Feature, Error 0.6
skl_tree  0.847
forward   0.893 *min number of instances in leaf no work
post_hoc  0.913
forest10  0.91
forest100 0.914
1 Feature, Error 0.8
skl_tree  0.798
forward   0.88 *min number of instances in leaf no work
post_hoc  0.886
forest10  0.916
forest100 0.923
1 Feature, Error 1
skl_tree  0.681
forward   0.841 *min number of instances in leaf no work
post_hoc  0.896
forest10  0.918
forest100 0.921
1 Feature, Error 1.2
skl_tree  0.611
forward   0.788 *min number of instances in leaf no work
post_hoc  0.867
forest10  0.909
forest100 0.917
1 Feature, Er

In [33]:
from ipywidgets import IntProgress
from IPython.display import display
import time
from IPython.core.display import HTML

# Custom CSS to change the background color of the progress bar
display(HTML("""
    <style>
        .progress .progress-bar {
            background-color: #f60 !important; /* Change the progress bar background color to black */
        }
    </style>
"""))

for i in range(3):
    print(i)
    max_count = 100

    f = IntProgress(min=0, max=max_count)  # instantiate the bar
    display(f)  # display the bar

    count = 0
    while count <= max_count:
        f.value += 1  # signal to increment the progress bar
        time.sleep(0.1)
        count += 1

0


IntProgress(value=0)

1


IntProgress(value=0)

2


IntProgress(value=0)

In [41]:
from tqdm.notebook import tqdm
a = tqdm(total=100, desc="Feature")
b = tqdm(total=100, desc="Feature")
for i in range(100):
    b.reset()
    for i in range(100):
        time.sleep(.01)
        b.update()
    a.update()

Feature:   0%|          | 0/100 [00:00<?, ?it/s]

Feature:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 